In [1]:
# نصب TensorFlow در صورت نیاز
!pip install tensorflow

# اتصال به Google Drive
from google.colab import drive
drive.mount('/content/drive')

# مسیر دیتاست
train_dir = '/content/drive/MyDrive/Divar Image/train'
test_dir = '/content/drive/MyDrive/Divar Image/test'

# پیش‌پردازش با Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical'
)

# Attention Block
import tensorflow as tf
from tensorflow.keras import layers

def attention_block(inputs):
    attention = layers.GlobalAveragePooling2D()(inputs)
    attention = layers.Reshape((1, 1, -1))(attention)
    attention = layers.Dense(inputs.shape[-1], activation='sigmoid')(attention)
    return layers.Multiply()([inputs, attention])

# Bilinear Pooling
def bilinear_pooling(x):
    x1 = tf.reshape(x, [tf.shape(x)[0], -1, x.shape[-1]])
    x2 = tf.transpose(x1, perm=[0, 2, 1])
    phi_I = tf.matmul(x2, x1) / tf.cast(tf.shape(x1)[1], tf.float32)
    phi_I = tf.reshape(phi_I, [tf.shape(x)[0], -1])
    phi_I = tf.math.sign(phi_I) * tf.math.sqrt(tf.abs(phi_I) + 1e-12)
    phi_I = tf.nn.l2_normalize(phi_I, axis=-1)
    return phi_I

def bilinear_pooling_shape(input_shape):
    dim = input_shape[-1]
    return (input_shape[0], dim * dim)

# ساخت مدل با Fine-tuning و Attention
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Lambda
from tensorflow.keras.optimizers import Adam

num_classes = 5

input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

# آزادسازی لایه‌های بالایی برای Fine-tuning
base_model.trainable = True

features = base_model.output
attended = attention_block(features)

bcnn = Lambda(bilinear_pooling, output_shape=bilinear_pooling_shape)(attended)
drop = Dropout(0.5)(bcnn)
output = Dense(num_classes, activation='softmax')(drop)

model = Model(inputs=input_tensor, outputs=output)

# کامپایل مدل با learning rate پایین
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# ذخیره بهترین مدل
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/best_fgic_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# آموزش مدل
model.fit(train_generator, validation_data=test_generator, epochs=30, callbacks=[checkpoint])

# ارزیابی نهایی
loss, acc = model.evaluate(test_generator)
print(f"Test Accuracy: {acc * 100:.2f}%")


Mounted at /content/drive
Found 2418 images belonging to 5 classes.
Found 1462 images belonging to 5 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.3157 - loss: 1.5954 
Epoch 1: val_accuracy improved from -inf to 0.18741, saving model to /content/drive/MyDrive/best_fgic_model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 2813s 36s/step - accuracy: 0.3167 - loss: 1.5952 - val_accuracy: 0.1874 - val_loss: 1.6095
Epoch 2/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 574ms/step - accuracy: 0.5231 - loss: 1.4515
Epoch 2: val_accuracy improved from 0.18741 to 0.19357, saving model to /content/drive/MyDrive/best_fgic_model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 71s 903ms/step - accuracy: 0.5236 - loss: 1.4508 - val_accuracy: 0.1936 - val_loss: 1.6090
Epoch 3/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 775ms/step - accuracy: 0.6881 - loss: 1.1962
Epoch 3: val_accuracy improved from 0.19357 to 0.20725, saving model to /content/drive/MyDrive/best_fgic_model.h5


...

76/76 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9827 - loss: 0.1012 - val_accuracy: 0.9945 - val_loss: 0.0641
Epoch 17/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 753ms/step - accuracy: 0.9836 - loss: 0.0965
Epoch 17: val_accuracy improved from 0.99453 to 0.99658, saving model to /content/drive/MyDrive/best_fgic_model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 978ms/step - accuracy: 0.9836 - loss: 0.0965 - val_accuracy: 0.9966 - val_loss: 0.0503
Epoch 18/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 754ms/step - accuracy: 0.9868 - loss: 0.0770
Epoch 18: val_accuracy did not improve from 0.99658
76/76 ━━━━━━━━━━━━━━━━━━━━ 63s 832ms/step - accuracy: 0.9868 - loss: 0.0770 - val_accuracy: 0.9966 - val_loss: 0.0435
Epoch 19/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 571ms/step - accuracy: 0.9909 - loss: 0.0633
Epoch 19: val_accuracy improved from 0.99658 to 0.99795, saving model to /content/drive/MyDrive/best_fgic_model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 85s 873ms/step - accuracy: 0.9909 - loss: 0.0634 - val_accuracy: 0.9979 - val_loss: 0.0381
Epoch 20/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 756ms/step - accuracy: 0.9889 - loss: 0.0612
Epoch 20: val_accuracy did not improve from 0.99795
76/76 ━━━━━━━━━━━━━━━━━━━━ 64s 836ms/step - accuracy: 0.9889 - loss: 0.0613 - val_accuracy: 0.9966 - val_loss: 0.0349
Epoch 21/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 559ms/step - accuracy: 0.9896 - loss: 0.0578
Epoch 21: val_accuracy improved from 0.99795 to 0.99863, saving model to /content/drive/MyDrive/best_fgic_model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 61s 809ms/step - accuracy: 0.9896 - loss: 0.0579 - val_accuracy: 0.9986 - val_loss: 0.0298
Epoch 22/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 579ms/step - accuracy: 0.9888 - loss: 0.0555
Epoch 22: val_accuracy did not improve from 0.99863
76/76 ━━━━━━━━━━━━━━━━━━━━ 75s 716ms/step - accuracy: 0.9888 - loss: 0.0555 - val_accuracy: 0.9986 - val_loss: 0.0273
Epoch 23/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 557ms/step - accuracy: 0.9901 - loss: 0.0485
Epoch 23: val_accuracy did not improve from 0.99863
76/76 ━━━━━━━━━━━━━━━━━━━━ 49s 637ms/step - accuracy: 0.9900 - loss: 0.0486 - val_accuracy: 0.9973 - val_loss: 0.0268
Epoch 24/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 566ms/step - accuracy: 0.9937 - loss: 0.0421
Epoch 24: val_accuracy did not improve from 0.99863
76/76 ━━━━━━━━━━━━━━━━━━━━ 49s 644ms/step - accuracy: 0.9937 - loss: 0.0421 - val_accuracy: 0.9986 - val_loss: 0.0214
Epoch 25/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 576ms/step - accuracy: 0.9934 - loss: 0.0410
Epoch 25: val_accuracy d

76/76 ━━━━━━━━━━━━━━━━━━━━ 64s 842ms/step - accuracy: 0.9912 - loss: 0.0416 - val_accuracy: 1.0000 - val_loss: 0.0160
Epoch 28/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 782ms/step - accuracy: 0.9897 - loss: 0.0366
Epoch 28: val_accuracy did not improve from 1.00000
76/76 ━━━━━━━━━━━━━━━━━━━━ 66s 867ms/step - accuracy: 0.9897 - loss: 0.0366 - val_accuracy: 0.9993 - val_loss: 0.0161
Epoch 29/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 584ms/step - accuracy: 0.9943 - loss: 0.0284
Epoch 29: val_accuracy did not improve from 1.00000
76/76 ━━━━━━━━━━━━━━━━━━━━ 50s 658ms/step - accuracy: 0.9943 - loss: 0.0284 - val_accuracy: 0.9993 - val_loss: 0.0132
Epoch 30/30
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 579ms/step - accuracy: 0.9946 - loss: 0.0255
Epoch 30: val_accuracy did not improve from 1.00000
76/76 ━━━━━━━━━━━━━━━━━━━━ 55s 716ms/step - accuracy: 0.9945 - loss: 0.0255 - val_accuracy: 0.9993 - val_loss: 0.0123
46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step - accuracy: 0.9983 - loss: 0.0132
Test Accuracy: 99.93%
